**Disclaimer:**  
This program was developed as part of the Programming assignment for the Artificial Intelligence Introduction course by:  
- Filza Rahma Muflihah (1301201261)  
- Ummu Husnul Khatimah (1301204120)  

**Instructions for Use:**  
1. **Using in Google Colab**  
   - Upload the 'bengkel.xlsx' file before running the program.  
   - To upload, click the folder icon (fourth from the top left), then use the upload option on the far left.  

2. **Running Locally**  
   - Ensure Python and the required libraries are installed on your system.  
   - Place the 'bengkel.xlsx' file in the same directory as the program file.  
   - Run the program using your preferred Python IDE or from the command line.  

Thank you for using this program!

In [1]:
# Importing Required Libraries
import pandas as pd

Data from Excel:      id  servis  harga
0     1      58      7
1     2      54      1
2     3      98      2
3     4      52      4
4     5      11      4
..  ...     ...    ...
95   96      30      1
96   97      25      3
97   98      27     10
98   99       8      6
99  100      11      8

[100 rows x 3 columns]


In [ ]:
# Step 1: Reading the Input Excel File
# Load the data from the Excel file into a pandas DataFrame for easier processing.
# Additionally, convert the data into a list of dictionaries for row-by-row access.
tabel = pd.read_excel('bengkel.xlsx', sheet_name='Sheet1')
data = tabel.to_dict('records')
print("Data from Excel:", tabel)

In [2]:
# Step 2: Fuzzification Functions
# Define a function to fuzzify the service quality score.
# This function maps the input score into linguistic variables like "buruk" (bad),
# "cukup" (fair), and "bagus" (good) along with their membership degrees.
def fuzzify_service(x):
    dict_servis = {}
    if x <= 33:
        # If the score is less than or equal to 33, it is fully "buruk" (bad).
        dict_servis[1] = ['buruk', -(x-33)/(33-0)]
        dict_servis[2] = ['buruk', (x-0)/(33-0)]
    elif x > 33 and x <= 50:
        # In the range 34 to 50, it transitions from "buruk" to "cukup" (fair).
        dict_servis[1] = ['buruk', -(x-50)/(50-33)]
        dict_servis[2] = ['cukup', (x-33)/(50-33)]
    elif x > 50 and x <= 70:
        # In the range 51 to 70, it transitions from "cukup" to "bagus" (good).
        dict_servis[1] = ['cukup', -(x-70)/(70-50)]
        dict_servis[2] = ['bagus', (x-50)/(70-50)]
    else:
        # If the score is above 70, it is fully "bagus" (good).
        dict_servis[1] = ['bagus', -(x-100)/(100-70)]
        dict_servis[2] = ['bagus', (x-70)/(100-70)]
    return dict_servis

# Define a function to fuzzify the price score.
# This function maps the input score into linguistic variables like "murah" (cheap)
# and "mahal" (expensive) along with their membership degrees.
def fuzzify_price(y):
    dict_harga = {}
    if y <= 5:
        # If the price is less than or equal to 5, it is fully "murah" (cheap).
        dict_harga[1] = ['murah', -(y-5)/(5-0)]
        dict_harga[2] = ['murah', (y-0)/(5-0)]
    elif y > 5 and y <= 8:
        # In the range 6 to 8, it transitions from "murah" to "mahal" (expensive).
        dict_harga[1] = ['murah', -(y-8)/(8-5)]
        dict_harga[2] = ['mahal', (y-5)/(8-5)]
    else:
        # If the price is above 8, it is fully "mahal" (expensive).
        dict_harga[1] = ['mahal', -(y-10)/(10-8)]
        dict_harga[2] = ['mahal', (y-8)/(10-8)]
    return dict_harga


In [3]:
# Step 3: Inference
# Define a function to perform inference using fuzzy rules.
# This function determines whether the service-price combination is "recommended"
# or "not recommended" based on the fuzzy membership values.
def inference(ds, dh):
    hasil = {'not recommended': 0, 'recommended': 0}  # Initialize results
    for i in range(1, 3):  # Loop over service fuzzy values
        for j in range(1, 3):  # Loop over price fuzzy values
            if ds[i][0] == 'buruk':  # If service is "buruk", it's "not recommended"
                infer = 'not recommended'
            elif ds[i][0] == 'cukup':  # If service is "cukup"
                infer = 'recommended' if dh[j][0] == 'murah' else 'not recommended'
            else:  # If service is "bagus", it's always "recommended"
                infer = 'recommended'

            # The membership value for the rule is the minimum of the two memberships
            nilai = min(ds[i][1], dh[j][1])
            if nilai > hasil[infer]:  # Update the result if the value is higher
                hasil[infer] = nilai
    return hasil

In [4]:
# Step 4: Defuzzification
# Define a function to defuzzify the results into a crisp value.
# This function calculates a weighted average based on the membership degrees
# of "recommended" and "not recommended".
def defuzzify(output):
    numerator = (output['not recommended'] * 50) + (output['recommended'] * 100)
    denominator = output['not recommended'] + output['recommended']
    return numerator / denominator if denominator != 0 else 0  # Avoid division by zero

In [5]:
# Step 5: Processing the Data
# Loop through each row in the data, fuzzify the inputs, apply inference,
# and calculate a crisp score for ranking.
tab_dic = []
for record in data:
    # Fuzzification
    ds = fuzzify_service(record['servis'])
    dh = fuzzify_price(record['harga'])

    # Inference
    f_output = inference(ds, dh)

    # Defuzzification
    crisp = defuzzify(f_output)

    # Store the Results
    tab_dic.append({'rank': 0, 'id': record['id'], 'z*': crisp})

In [6]:
# Step 6: Ranking
# Sort the results by the crisp score in descending order and assign rankings.
tab_dic = sorted(tab_dic, key=lambda x: x['z*'], reverse=True)
for rank, item in enumerate(tab_dic, start=1):
    item['rank'] = rank

In [7]:
# Step 7: Top 10 Results
# Extract the top 10 entries based on the ranking.
top_10 = tab_dic[:10]
print("Top 10 Results:", top_10)

Top 10 Results: [{'rank': 1, 'id': 2, 'z*': 100.0}, {'rank': 2, 'id': 3, 'z*': 100.0}, {'rank': 3, 'id': 4, 'z*': 100.0}, {'rank': 4, 'id': 13, 'z*': 100.0}, {'rank': 5, 'id': 15, 'z*': 100.0}, {'rank': 6, 'id': 16, 'z*': 100.0}, {'rank': 7, 'id': 17, 'z*': 100.0}, {'rank': 8, 'id': 22, 'z*': 100.0}, {'rank': 9, 'id': 24, 'z*': 100.0}, {'rank': 10, 'id': 31, 'z*': 100.0}]


In [9]:
# Step 8: Saving the Results to Excel
# Save the top 10 results to an Excel file for further analysis or reporting.
pd.DataFrame(top_10).to_excel('peringkat.xlsx', index=False)
print("Top 10 saved to 'peringkat.xlsx'")

Top 10 saved to 'peringkat.xlsx'
